In [ ]:
# Loading libraries
import pandas as pd

In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# loading datasets
train_df = pd.read_csv("/content/drive/MyDrive/CFG_Project/fraudTrain.csv")
test_df = pd.read_csv("/content/drive/MyDrive/CFG_Project/fraudTest.csv")

In [ ]:
# merging pre-split training and test datasets into one dataframe
df = train_df.append(test_df, ignore_index=True)

In [ ]:
df.head(3)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0


In this section I am exploring the dataset to learn more about its content and characteristics. My priorities are ensuring I understand what each column represents, the datatypes I will be manipulating, check if I need to handle missing values and other anomalies and identify whether there is irrelevant data that I will not need to take forward to build the anti-fraud predictive model. 

In [ ]:
df.shape

(1852394, 23)

In [ ]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

Since the documentation shared on Kaggle about the dataset does not provide further details about the overall dataset structure, I am engaging with the content of each column to familiarise myself with what the data recorded in it represents. I am immediately noticing that specific columns will not be relevant in future analysis related to fraud prediction, for example: 'cc_num' (unique credit card number), 'merchant' (unique merchant first and last name), 'first' (credit card owner first name), 'last' (credit card owner last name), 'trans_num' (unique transaction number). We do not want the model to develop bias related to specific names or unique card/transaction numbers so it would be actively harmful to take forward this data. In addition, the 'trans_date_trans_time' column might not be necessary since the same data is also recorded in the 'unix_time' column, however it is better to keep in mind it exists, in case we might need it during the pre-processing phase. For the time being however, I will treat 'trans_date_trans_time' as duplicate data.  

In [ ]:
# Checking datatypes

df.dtypes

Unnamed: 0                 int64
trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object

In [ ]:
# The first thing I want to verify is the ratio of fraudulent:genuine transactions.
# The ratio is about 0.005:1; the vast majority of our dataset is comprised of genuine transactions.
# In other words, our dataset is unbalanced, which might create issues with overfitting once we start training the model
 
print(df['is_fraud'].value_counts())

0    1842743
1       9651
Name: is_fraud, dtype: int64


In [ ]:
print(df['category'].describe())

count           1852394
unique               14
top       gas_transport
freq             188029
Name: category, dtype: object


In [ ]:
print(df['gender'].value_counts())

F    1014749
M     837645
Name: gender, dtype: int64


In [ ]:
print(df['street'].describe())

count             1852394
unique                999
top       444 Robert Mews
freq                 4392
Name: street, dtype: object


In [ ]:
print(df['city'].describe())

count        1852394
unique           906
top       Birmingham
freq            8040
Name: city, dtype: object


In [ ]:
print(df['state'].describe())

count     1852394
unique         51
top            TX
freq       135269
Name: state, dtype: object


In [ ]:
print(df['zip'].value_counts())

82514    5116
73754    5116
48088    5115
34112    5108
61454    4392
         ... 
38915       7
61364       7
37411       7
95688       7
52658       6
Name: zip, Length: 985, dtype: int64


In [ ]:
print(df['lat'].value_counts())

36.3850    5116
43.0048    5116
42.5164    5115
26.1184    5108
40.6761    4392
           ... 
35.0271       7
41.1225       7
41.1165       7
38.3847       7
40.7067       6
Name: lat, Length: 983, dtype: int64


In [ ]:
print(df['job'].describe())

count               1852394
unique                  497
top       Film/video editor
freq                  13898
Name: job, dtype: object


In [ ]:
print(df['dob'].value_counts())

1977-03-23    8044
1988-09-15    6574
1981-08-29    6571
1955-05-06    5121
1960-01-13    4395
              ... 
1944-06-17       7
1932-05-09       7
1936-05-04       7
1963-02-26       7
1998-08-02       6
Name: dob, Length: 984, dtype: int64


For the time being I am not considering the following input variables:
1. 'street' --it has a very large pool of unique values, I am considering excluding it altogether, especially considering that 'state' is already giving us info about the geographical location.
2. 'city' --it has a very large pool of unique values, I am considering excluding it altogether, especially considering that 'state' is already giving us info about the geographical location.
3. 'lat', 'long', 'merch_lat', 'merch_long' --it is data about the specific geographical position, I am unsure how to handle it at the moment. Moreover, like 'city' and 'street' it has a large pool of unique values.
4. 'zip' --altough this is one of the input variables already available as an integer, I don't want to include it into the analysis since it is not data tied by an ordinal relationship. 

--if I had more time I would harvest the geographical data better.

In [ ]:
# Checking for missing values
# Since we are using synthetic data we do not need to handle incorrect, missing or invalid data

df.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

Here I am proceeding to the pre-processing phase. I will be experimenting with using OneHotEncoder to transform categoric data into numerical data so that a model can more easily process it. I have chosen this encoding method since our categoric data does not have a pre-established order. For example, if we were to assign 1 to a specific individual credit card owner's job title and 2 to a different job title, this would train the model to consider these values as somehow relevant to the analysis. In other words, the fact that the latter is a larger quantity than the former would be interpreted as significative, when the numerical values assigned to each job title are in fact random. Since I am not able to convert this categoric data into numerical values that express a meaninfgul ordinal relationship, using OneHotEncoder is the appropriate choice.   

In [ ]:
#Removing irrelevant variables
irrelevantVar_list = ["trans_date_trans_time", 
                      "first", 
                      "last", 
                      "street", 
                      "lat", 
                      "long",  
                      "cc_num", 
                      "trans_num", 
                      "city", 
                      "zip", 
                      "merchant", 
                      "merch_lat", 
                      "merch_long"]
                      
df.drop(irrelevantVar_list, axis=1, inplace=True)

In [ ]:
# Here I am starting with the input variable 'gender'
# First, I am creating a new dataframe that includes a copy of all data related to categorical features

obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

,category,gender,state,job,dob
0,misc_net,F,NC,"Psychologist, counselling",1988-03-09
1,grocery_pos,F,WA,Special educational needs teacher,1978-06-21
2,entertainment,M,ID,Nature conservation officer,1962-01-19
3,gas_transport,M,MT,Patent attorney,1967-01-12
4,misc_pos,M,VA,Dance movement psychotherapist,1986-03-28


In [ ]:
# Employing OneHotEncoder means that we are adding a new column 
# for each unique value associated with the input variable we are encoding
# since 'gender' only has F or M as unique values that need to be encoded, it adds only 2 new columns to the df

from sklearn.preprocessing import OneHotEncoder
oe_gender = OneHotEncoder()
oe_gender_results = oe_gender.fit_transform(obj_df[["gender"]])
pd.DataFrame(oe_gender_results.toarray(), columns=oe_gender.categories_).head()

,F,M
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [ ]:
# Here I am joining the resulting encoded columns on the dataframe containing all categorical data
# that I created previously 

obj_df = obj_df.join(pd.DataFrame(oe_gender_results.toarray(), columns=oe_gender.categories_))

In [ ]:
obj_df.head(5)

,category,gender,state,job,dob,"(F,)","(M,)"
0,misc_net,F,NC,"Psychologist, counselling",1988-03-09,1.0,0.0
1,grocery_pos,F,WA,Special educational needs teacher,1978-06-21,1.0,0.0
2,entertainment,M,ID,Nature conservation officer,1962-01-19,0.0,1.0
3,gas_transport,M,MT,Patent attorney,1967-01-12,0.0,1.0
4,misc_pos,M,VA,Dance movement psychotherapist,1986-03-28,0.0,1.0


In [ ]:
# Now I will try something more complex to encode: the state input variable
# This has a much larger pool of values so it will create a lot of additional columns

oe_state = OneHotEncoder()
oe_state_results = oe_state.fit_transform(obj_df[["state"]])
pd.DataFrame(oe_state_results.toarray(), columns=oe_state.categories_).head()

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
obj_df = obj_df.join(pd.DataFrame(oe_state_results.toarray(), columns=oe_state.categories_))

In [ ]:
# Here I will be encoding the 'category' categorical feature

oe_category = OneHotEncoder()
oe_category_results = oe_category.fit_transform(obj_df[["category"]])
pd.DataFrame(oe_category_results.toarray(), columns=oe_category.categories_).head()

,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
obj_df = obj_df.join(pd.DataFrame(oe_category_results.toarray(), columns=oe_category.categories_))

In [ ]:
# I attempted to perform the same process to encode the categorical feature 'job', 
# however it resulted in almost 500 columns, an extremely sparse matrix.
# This lead to the decision to drop this feature, 
# however with more resources at my disposal I would have handled this differently.

# oe_state = OneHotEncoder()
# oe_state_results = oe_state.fit_transform(obj_df[["state"]])
# pd.DataFrame(oe_state_results.toarray(), columns=oe_state.categories_).head()
# obj_df = obj_df.join(pd.DataFrame(oe_state_results.toarray(), columns=oe_state.categories_))


In [ ]:
# Here I am appending the dataframe containing all encoded categorical values to the original dataframe

t_df = obj_df.append(df, ignore_index=True)

In [ ]:
t_df.head(5)

,category,gender,state,job,dob,"(F,)","(M,)","(AK,)","(AL,)","(AR,)",...,"(misc_pos,)","(personal_care,)","(shopping_net,)","(shopping_pos,)","(travel,)",Unnamed: 0,amt,city_pop,unix_time,is_fraud
0,misc_net,F,NC,"Psychologist, counselling",1988-03-09,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,grocery_pos,F,WA,Special educational needs teacher,1978-06-21,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,entertainment,M,ID,Nature conservation officer,1962-01-19,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,gas_transport,M,MT,Patent attorney,1967-01-12,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,misc_pos,M,VA,Dance movement psychotherapist,1986-03-28,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# The resulting dataframe has 77 columns

t_df.shape

(3704788, 77)

In [ ]:
# I am now dropping the columns containing the original categorical data, so that only their encoded version remains

Encoded_List = ['category', 'gender', 'state']
t_df.drop(Encoded_List, axis=1, inplace=True)

In [ ]:
job_variable = ['job']
t_df.drop(job_variable, axis=1, inplace=True)

I'm on my way for the pre-processing pipeline but there are several challenged to consider. These are the issues I am currently looking at:
 
1. For the time being I managed to use OneHotEncoder to encode 'gender', 'state', 'category', 'job'. Unfortunately I have done this manually, I am still experiementing with creating a pipeline that is just applied to the full dataset. 
3. I have also had some issues merging the dataframes, while I can get the original dataset to show up on the same dataframe as the encoded gender, state and city, I am not managing to also add the encoded job dataframe, it is unclear whether the number of columns has an impact on this (encoded job dataframe has over 400 columns, the others have less than 100 between all of them).  
4. For the time being I am leaving aside the following features, here is the list and ideas I have on how I'm going to deal with them, however further research is still required:

1. 'trans_date_trans_time' --treated as duplicate data for the time being, I will be using unix_time input variable instead. 
2. 'first', 'last' --I am not considering individual names as relevant variables, they are unique values, and when they are not (such as repeated first/last), considering them relevant could lead to discriminating against specific individuals.
3. 'merchant' --I am not considering individual names as relevant variables.
4. 'street' --it has a very large pool of unique values, I am considering excluding it altogether, as state is already giving us relevant information about the geographical location, or group it in some way.
5. 'city --it has a very large pool of unique values, might need to group it in some way to create classes and then encode those (perhaps regions to be more specific than states?).
6. 'lat', 'long', 'merch_lat', 'merch_long' --it is data about geographical positions, probably needs to be encoded differently.
'dob' --I am considering converting into year only and encode that (possibly age ranges?).